# Google Colab with Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




---



#**TextMining Team2 - 오늘 뭐하지?**

구글 드라이브 링크 -> 공유 폴더를 '드라이브에 바로가기 추가' 하신 뒤 Run ALL 하시면 작동합니다!

(모델을 불러오는 과정에서 시간이 조금 소요됩니다)

**※ 지원하는 지역 리스트 : [송파구, 중구, 종로구, 용산구, 강남구, 서초구, 서대문구, 동대문구, 마포구, 영등포구, 성동구, 용산구, 광진구]**

# PATH

In [2]:
# 경로 설정 후 Run all 해주시면 맨 아래 쉘의 코드가 실행됩니다.

# 데이터 파일(Dataset)의 경로를 설정해주세요
DATA_PATH = '/content/drive/MyDrive/[Team2] TextMining - 오늘 뭐하지?/dataset.csv'

# 지하철 데이터 파일(Metro)의 경로를 설정해주세요
METRO_PATH = '/content/drive/MyDrive/[Team2] TextMining - 오늘 뭐하지?/지하철_좌표.csv'

# 모델 파일의 경로를 설정해주세요
MODEL_PATH = '/content/drive/MyDrive/[Team2] TextMining - 오늘 뭐하지?/FastTextModel.bin'

# Install

In [3]:
!pip install nltk
!pip install gensim
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 36.5 MB/s eta 0:00:00


# Import

In [4]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

# Random Seed fix

In [5]:
import random as python_random # 시드값 고정
seed_num = 42
np.random.seed(seed_num)
python_random.seed(seed_num)

# Data Import

In [6]:
metro = pd.read_csv(METRO_PATH)
data = pd.read_csv(DATA_PATH)
data = data.fillna('')

food = data[data['구분']=='식당']
place = data[data['구분']=='장소']
cafe = data[data['구분']=='카페']
bar =  data[data['구분']=='술집']

# Model Import

In [7]:
# 모델 Import에 시간이 조금 걸립니다.
from gensim.models import fasttext

model = fasttext.load_facebook_model(MODEL_PATH, encoding='utf-8')

### Model의 유사 키워드 추출 (중간결과)

In [9]:
'''

# 유사 키워드 추출

word = "피자"
sim_words_list = model.wv.most_similar([word], topn=100)

print(sim_words_list[:10])
print(sim_words_list[40:50])

'''

[('피자로', 0.9249041080474854), ('피자집', 0.9222275018692017), ('피자가', 0.9129948616027832), ('피자도', 0.9122931361198425), ('치즈피자', 0.9057206511497498), ('피자가게가', 0.904869556427002), ('반반피자', 0.892049252986908), ('피자와', 0.891700029373169), ('포테이토피자', 0.8916018605232239), ('마늘빵엣지스테이크피자', 0.8908000588417053)]
[('치즈떡볶이', 0.8432779312133789), ('아이스크림으로', 0.8428207635879517), ('아이스크림이다', 0.8425928950309753), ('이태리식', 0.8417824506759644), ('레스토랑이다', 0.8417503833770752), ('매콤크림파스타', 0.8415462970733643), ('미스터피자는', 0.8412715196609497), ('더블갈릭바베큐피자', 0.8409932851791382), ('치킨감자', 0.8407616019248962), ('고르곤졸라피자', 0.8405730128288269)]


# 모델을 이용한 데이터셋 임베딩

In [8]:
# 모델을 이용한 데이터셋의 임베딩

def add_vectorized_column(df, model):
    # '특성' 열의 값을 벡터화하여 'main_vec' 열 추가
    df['main_vec'] = df['특성'].apply(lambda x: [model.wv[word] for word in x if word in model.wv])
    df['doc_vec'] = df['doc_key'].apply(lambda x: [model.wv[word] for word in x if word in model.wv])

    return df

food = add_vectorized_column(food, model)
cafe = add_vectorized_column(cafe, model)
place = add_vectorized_column(place, model)
bar = add_vectorized_column(bar, model)

# 사용자 입력문장 키워드 추출 함수

In [9]:
from konlpy.tag import Okt

def extract_keywords(sentence):
    # 형태소 분석기 인스턴스 생성
    okt = Okt()

    # 형태소 분석 수행
    morphemes = okt.pos(sentence, stem=True)

    # 부사, 형용사, 명사 추출
    keywords_main = [word for word, tag in morphemes if tag in ('Noun')]
    keywords_sub = [word for word, tag in morphemes if tag in ('Adjective', 'Adverb')]

    # 제외하고 싶은 단어들
    exclude_words_main = ["나", "오늘", "가다", "먹다", "곳", "식당", "카페", '술', '없다', '어떻다', '거나', '음식', '이면', '추천', '걸','건', '것', '땡기다', '약속', '장소', '출발', '안녕','뭔가', '아침', '점심', '저녁','간식','식사','가게','종류','류','게']

    # 제외할 동사 및 조사 설정
    exclude_verbs_and_josa = ['빼다', '제외하다', '말고', '빼고', '싫다', '싫어', '싫고']

    # ~비슷한, ~같은의 키워드가 있는가?
    Islike = False

    # 이후 검색에서 제외할 키워드
    exclude_keywords = []

    # '같다' 또는 '비슷하다' 키워드가 있으면 Islike를 True로 설정하고 키워드 제거
    for i, (word, tag) in enumerate(morphemes):
        if word in ['같다', '같이', '비슷하다']:
            Islike = True
            # 이전 단어가 명사, 형용사, 부사인 경우 제외 리스트에 추가
            j = i - 1
            while j >= 0:
                if morphemes[j][1] in ('Noun', 'Adjective', 'Adverb'):
                    exclude_words_main.append(morphemes[j][0])
                    exclude_keywords.append(morphemes[j][0])
                    break
                j -= 1

    # 동사 및 조사를 찾아 그 앞에 있는 명사, 형용사, 부사 제외
    for i, (word, tag) in enumerate(morphemes):
        if word in exclude_verbs_and_josa:
            j = i - 1
            # 명사, 형용사, 부사 찾기
            while j >= 0:
                if morphemes[j][1] in ('Noun', 'Adjective', 'Adverb'):
                    exclude_words_main.append(morphemes[j][0])
                    exclude_keywords.append(morphemes[j][0])
                    break
                j -= 1

    # 제외된 단어들을 제외한 새로운 리스트 생성
    keywords_main = [word for word in keywords_main if word not in exclude_words_main]
    keywords_sub = [word for word in keywords_sub if word not in exclude_words_main and word not in exclude_verbs_and_josa]
    exclude_keywords = [word for word in exclude_keywords if word not in ['비슷하다', '같다']]

    return keywords_main, keywords_sub, exclude_keywords, Islike


### 키워드 추출 예시 (중간 결과)

In [12]:


# 위의 키워드 추출 결과 확인을 위한 쉘
# sentence = input("문장을 입력하세요: ")
sentence = '나는 오늘 달콤한게 먹고 싶어. 도넛은 빼고'
keywords_main, keywords_sub, exclude_keywords, Islike = extract_keywords(sentence)

print("추출된 주요 키워드 :", keywords_main)
print("추출된 추상 키워드 (형용사, 부사) :", keywords_sub)
print("제외 키워드 :", exclude_keywords)
print("Islike :", Islike)



문장을 입력하세요: 나는 오늘 달콤한게 먹고 싶어. 도넛은 빼고
추출된 주요 키워드 : []
추출된 추상 키워드 (형용사, 부사) : ['달콤하다']
제외 키워드 : ['도넛']
Islike : False


# 추천 함수

In [13]:
from geopy.distance import geodesic
from sklearn.metrics.pairwise import cosine_similarity

def calculate_distance(row, loc_x, loc_y):
    return geodesic((loc_x, loc_y), (row['위도'], row['경도'])).kilometers


def calculate_similarity(a, b):
    return cosine_similarity([a], [b])[0][0]


def calculate_average_top_similarities(word_vec, key_vec):
    similarities = []
    for w in word_vec:
        for k in key_vec:
            similarities.append(calculate_similarity(w, k))
    similarities.sort(reverse=True)
    top_similarities = similarities[:5]
    return np.mean(top_similarities)


def recommend_rows_based_on_keywords(df, model, keywords_main, keywords_sub, exclude_keywords, Islike, loc_x, loc_y):
    # 단계 2: loc_x, loc_y와의 직선 거리가 1.5km 이내인 행으로 tmp_df
    df['distance'] = df.apply(calculate_distance, args=(loc_x, loc_y), axis=1)

    # 거리가 1.5km 이내인 행만 선택
    tmp_df = df[df['distance'] <= 3].copy()

    # 단계 3: keywords_main으로 main_word_vec 생성
    main_word_vec = [model.wv[word] for word in keywords_main if word in model.wv]

    # 단계 4: Islike가 True인 경우, sim_words_list 생성 후 sim_word_vec 생성
    if (keywords_main==[]) : Islike =False
    if Islike  :
        sim_words = model.wv.most_similar(positive=keywords_main, topn=50)
        sim_words_list = [word for word, similarity in sim_words[10:15]] + [word for word, similarity in sim_words[40:45]]
        sim_word_vec = [model.wv[word] for word in sim_words_list if word in model.wv]

    # 단계 6: exclude_keywords가 빈 리스트가 아닌 경우, except_word_vec 생성
    if exclude_keywords:
        except_word_vec = [model.wv[word] for word in exclude_keywords if word in model.wv]

    # 단계 7: keywords_sub가 빈 리스트가 아닌 경우, sub_word_vec 생성
    if keywords_sub:
        sub_word_vec = [model.wv[word] for word in keywords_sub if word in model.wv]

    # 단계 9: main_word_vec과 df의 'main_vec' column 사이의 코사인 유사도 측정
    main_sim_list = tmp_df['main_vec'].apply(lambda x: calculate_average_top_similarities(main_word_vec, x))

    # 단계 10: sub_word_vec이 존재하는 경우, sub_word_vec과 df의 'doc_vec' column 사이의 코사인 유사도 측정
    if keywords_sub:
        sub_sim_list = tmp_df['doc_vec'].apply(lambda x: calculate_average_top_similarities(sub_word_vec, x))

    # 단계 11: except_word_vec이 존재하는 경우
    if exclude_keywords:
        except_sim_list_1 = tmp_df['main_vec'].apply(lambda x: calculate_average_top_similarities(except_word_vec, x))
        except_sim_list_2 = tmp_df['doc_vec'].apply(lambda x: calculate_average_top_similarities(except_word_vec, x)) if keywords_sub else [0]*len(tmp_df)

    # 단계 12: sim_word_vec이 존재하는 경우 (Islike가 True인 경우)
    if Islike:
        sim_sim_list = tmp_df['main_vec'].apply(lambda x: calculate_average_top_similarities(sim_word_vec, x))

    # 단계 13: 리스트들 정렬하고 최종 점수 계산
    tmp_df['main_similarity'] = main_sim_list
    if keywords_sub:
        tmp_df['sub_similarity'] = sub_sim_list
    if exclude_keywords:
        tmp_df['except_similarity'] = [max(e1, e2) for e1, e2 in zip(except_sim_list_1, except_sim_list_2)]
    if Islike:
        tmp_df['sim_similarity'] = sim_sim_list


    # 최종 점수 계산
    if not keywords_main and not keywords_sub and not exclude_keywords:
        tmp_df = tmp_df.sample(frac=1).reset_index(drop=True)
    else:
      if keywords_sub and exclude_keywords and Islike:
           tmp_df['final_score'] = tmp_df['main_similarity']*1.7 + tmp_df['sub_similarity']*0.15 - tmp_df['except_similarity'] + tmp_df['sim_similarity']*0.15
      elif keywords_sub and exclude_keywords:
          tmp_df['final_score'] = tmp_df['main_similarity']*1.7 + tmp_df['sub_similarity']*0.3 - tmp_df['except_similarity']
      if keywords_sub and exclude_keywords and Islike:
          tmp_df['final_score'] = tmp_df['main_similarity']*1.7 + tmp_df['sub_similarity']*0.15 - tmp_df['except_similarity'] + tmp_df['sim_similarity']*0.15
      elif keywords_sub and exclude_keywords:
          tmp_df['final_score'] = tmp_df['main_similarity']*1.7 + tmp_df['sub_similarity']*0.3 - tmp_df['except_similarity']
      elif keywords_sub and Islike:
          tmp_df['final_score'] = tmp_df['main_similarity']*0.6 + tmp_df['sub_similarity']*0.2 + tmp_df['sim_similarity']*0.2
      elif exclude_keywords and Islike:
          tmp_df['final_score'] = tmp_df['main_similarity']*0.94 - tmp_df['except_similarity'] + tmp_df['sim_similarity']*1.06
      elif keywords_sub:
          tmp_df['final_score'] = tmp_df['main_similarity']*0.85 + tmp_df['sub_similarity']*0.15
      elif exclude_keywords:
          tmp_df['final_score'] = tmp_df['main_similarity'] - tmp_df['except_similarity']
      elif Islike:
          tmp_df['final_score'] = tmp_df['main_similarity']*0.47 + tmp_df['sim_similarity']*0.53
      else:
          tmp_df['final_score'] = tmp_df['main_similarity']

    # 단계 14, 15: 최종 추천
    result_df = tmp_df.sort_values(by='final_score', ascending=False)

    # 최종 추천된 데이터프레임 반환
    return result_df.head(10)

## 음식점 추천 항목 예시(중간 결과)

In [14]:
'''

# 추천 받을 주제의 키워드

keyword = '치킨'

word_vec = model.wv[[keyword]]

'''

In [15]:
'''

from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도 계산 함수
def calculate_similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

# 각 행에 대해 상위 5개 코사인 유사도의 평균을 계산하는 함수
def calculate_average_top_similarities(word_vec, key_vec):
    similarities = []
    for w in word_vec:
        for k in key_vec:
            similarities.append(calculate_similarity(w, k))
    similarities.sort(reverse=True)
    top_similarities = similarities[:5]
    return np.mean(top_similarities)

# 데이터프레임의 각 'key_vec'와의 코사인 유사도 계산
average_similarities = []
for index, row in food.iterrows():
    avg_similarity = calculate_average_top_similarities(word_vec, row['main_vec'])
    average_similarities.append((index, avg_similarity))

# 평균 유사도를 기준으로 내림차순 정렬
average_similarities.sort(key=lambda x: x[1], reverse=True)

# 상위 20개 결과 출력
top_20 = average_similarities[:20]
for index, avg_similarity in top_20:
    print(f"Index: {index}, Average Similarity: {avg_similarity}, Data: {food.loc[index]['장소명']}")

'''

Index: 62, Average Similarity: 0.7692724466323853, Data: 장원닭한마리
Index: 91, Average Similarity: 0.7692724466323853, Data: 칸나닭집
Index: 100, Average Similarity: 0.7692724466323853, Data: 큐스
Index: 879, Average Similarity: 0.7692724466323853, Data: 부지깽이
Index: 1089, Average Similarity: 0.7692724466323853, Data: 김종용누룽지통닭(한남점)
Index: 1187, Average Similarity: 0.7692724466323853, Data: 계림원
Index: 1198, Average Similarity: 0.7692724466323853, Data: 찜닭마루
Index: 1490, Average Similarity: 0.7692724466323853, Data: 두찜 노원하계점
Index: 1593, Average Similarity: 0.7692724466323853, Data: 일미리 금계찜닭 가재울점
Index: 1628, Average Similarity: 0.7692724466323853, Data: 밥꼬찜닭
Index: 1641, Average Similarity: 0.7692724466323853, Data: 동궁찜닭 서교점
Index: 1683, Average Similarity: 0.7692724466323853, Data: 두찜성동왕십리점
Index: 1832, Average Similarity: 0.7692724466323853, Data: 성식이네 매운닭발
Index: 1901, Average Similarity: 0.7692724466323853, Data: 오늘통닭(동대문역사문화공원역점)
Index: 2011, Average Similarity: 0.7692724466323853, Data: 두찜 신



---



# **- 오늘 뭐하지? -**

## 최종 실행 코드

In [16]:
import re

# 추천 결과
food_final = None
cafe_final = None
place_final = None
bar_final = None

flagFood = 1
flagCafe = 1
flagPlace = 1
flagBar = 1

def extract_stations(text):
    # "OO역" 또는 "OO OO역"을 찾는 정규 표현식
    pattern = re.compile(r'\b\w+(?:\s\w+)?역\b')

    # text에서 패턴에 일치하는 문자열 탐색
    matches = pattern.findall(text)

    # 띄어쓰기 제거 후 일치하는 문자열 반환
    stations = [match.replace(" ", "") for match in matches]

    # 일치하는 문자열 반환
    return stations[0]

def x_check(usrInput):
    if (usrInput == 'X' or usrInput == 'x'):
        return

def extract_integers(text):
    pattern = re.compile(r'\d+')  # 정수를 찾는 정규 표현식
    integers = pattern.findall(text)  # 문자열에서 정수 추출
    return [int(num) for num in integers]  # 추출된 정수들을 정수형으로 변환하여 반환

print("안녕하세요! 멋진 하루를 보낼 수 있도록 도와드리겠습니다!\n")
print("오늘 어디를 방문하실 계획인가요? 방문 계획이 있다면 인근 지하철 역을 말씀해주시겠어요? \n(지하철 역 입력을 원하지 않으시면 X를 입력해주세요)\n")
while(1):
  user_loc = input()
  if (user_loc == 'X' or user_loc == 'x'):
      user_loc = extract_stations('강남역')
      loc_x = metro[metro['역명']==user_loc]['위도'].values[0]
      loc_y = metro[metro['역명']==user_loc]['경도'].values[0]
      break
  else:
    try:
      user_loc = extract_stations(user_loc)
      loc_x = metro[metro['역명']==user_loc]['위도'].values[0]
      loc_y = metro[metro['역명']==user_loc]['경도'].values[0]
      break
    except:
      print("역 이름을 제대로 입력해주세요\n")



print("\n감사합니다! 그럼 이제 일정을 세워보겠습니다!")
print("금강산도 식후경! 우선 밥을 먹는 것은 어떨까요? 드시고 싶은 음식의 느낌을 자유롭게 적어주세요. \n(식당 추천을 원하지 않으시면 X를 입력해주세요)\n")

while(1):
  food_sentence = input()
  if (food_sentence == 'X' or food_sentence == 'x'):
      flagFood = 0
      break
  else:
    try:
      keywords_main, keywords_sub, exclude_keywords, Islike = extract_keywords(food_sentence)
      food_rec = recommend_rows_based_on_keywords(df=food, model=model, keywords_main=keywords_main, keywords_sub=keywords_sub, exclude_keywords=exclude_keywords, Islike=Islike, \
                                                  loc_x=loc_x, loc_y=loc_y)
      break
    except:
      print("죄송합니다. 문장을 이해할 수 없습니다. 다시 입력해주세요.\n")

if flagFood == 1:
  print("\n말씀해주신 결과를 바탕으로 다음과 같은 식당들을 찾아보았습니다! 마음에 드는 번호를 입력해주세요\n")

  place_name =[]
  for i in range(len(food_rec)):
      place_name = food_rec.iloc[i, :]['장소명']
      menu_name = food_rec.iloc[i, :]['대표메뉴'].split(',', 2)[:2]
      score = food_rec.iloc[i, :]['final_score']
      print("[{}번] 장소명: {}, 대표메뉴: {}".format(i+1, place_name, menu_name))
      i = i + 1

  if len(place_name) == 0 : # 추천지 중 거리 내에 있는 추천지가 없는 경우
    print('죄송합니다ㅠㅠ 조건을 만족하는 추천지가 없습니다!\n')

  else :
    while(1):
      try:
        food_selection = input()
        food_selection = extract_integers(food_selection)
        food_final = food_rec.iloc[food_selection[0] - 1,:]
        break
      except:
        print("제대로 된 번호를 입력해주세요! (예시: 3번)\n")

    loc_x = food_final['위도']
    loc_y = food_final['경도']


print("\n느낌있는 카페는 어떠신가요? 카페에 대해 자유롭게 적어주세요 \n(카페 추천을 원하지 않으시면 X를 입력해주세요)\n")

while(1):
  cafe_sentence = input()
  if (cafe_sentence == 'X' or cafe_sentence == 'x'):
      flagCafe = 0
      break
  else:
    try:
      keywords_main, keywords_sub, exclude_keywords, Islike = extract_keywords(cafe_sentence)
      cafe_rec = recommend_rows_based_on_keywords(df=cafe, model=model, keywords_main=keywords_main, keywords_sub=keywords_sub, exclude_keywords=exclude_keywords, Islike=Islike, \
                                                  loc_x=loc_x, loc_y=loc_y)
      break
    except:
      print("죄송합니다. 문장을 이해할 수 없습니다. 다시 입력해주세요.\n")

if flagCafe == 1:
    print("\n말씀해주신 결과를 바탕으로 다음과 같은 카페들을 찾아보았습니다! 마음에 드는 번호를 입력해주세요\n")

    place_name =[]
    for i in range(len(cafe_rec)):
        place_name = cafe_rec.iloc[i, :]['장소명']
        menu_name = cafe_rec.iloc[i, :]['대표메뉴'].split(',', 2)[:2]
        score = cafe_rec.iloc[i, :]['final_score']
        print("[{}번] 장소명: {}, 대표메뉴: {}".format(i+1, place_name, menu_name))
        i = i + 1
    if len(place_name) == 0 :
      print('죄송합니다ㅠㅠ 조건을 만족하는 추천지가 없습니다!\n')

    else :
      while(1):
        try:
          cafe_selection = input()
          cafe_selection = extract_integers(cafe_selection)
          cafe_final = cafe_rec.iloc[cafe_selection[0] - 1,:]
          break
        except:
          print("제대로 된 번호를 입력해주세요! (예시: 3번)\n")

      loc_x = cafe_final['위도']
      loc_y = cafe_final['경도']


print("\n재미있는 액티비티나 문화생활을 즐겨보실래요? 가고 싶은 장소에 대한 느낌을 자유롭게 적어주세요\n(장소 추천을 원하지 않으시면 X를 입력해주세요)\n")

while(1):
  place_sentence = input()
  if (place_sentence == 'X' or place_sentence == 'x'):
      flagPlace = 0
      break
  else:
    try:
      keywords_main, keywords_sub, exclude_keywords, Islike = extract_keywords(place_sentence)
      place_rec = recommend_rows_based_on_keywords(df=place, model=model, keywords_main=keywords_main, keywords_sub=keywords_sub, exclude_keywords=exclude_keywords, Islike=Islike, \
                                                  loc_x=loc_x, loc_y=loc_y)
      break
    except:
      print("죄송합니다. 문장을 이해할 수 없습니다. 다시 입력해주세요.\n")

if flagPlace == 1:
    print("\n말씀해주신 결과를 바탕으로 다음과 같은 장소들을 찾아보았습니다! 마음에 드는 번호를 입력해주세요\n")

    place_name =[]
    for i in range(len(place_rec)):
        place_name = place_rec.iloc[i, :]['장소명']
        menu_name = place_rec.iloc[i, :]['업종']
        score = place_rec.iloc[i, :]['final_score']

        print("[{}번] 장소명: {}, 업종: {}".format(i+1, place_name, menu_name))
        i = i + 1
    if len(place_name) == 0 :
      print('죄송합니다ㅠㅠ 조건을 만족하는 추천지가 없습니다!')

    else :
      while(1):
        try:
          place_selection = input()
          place_selection = extract_integers(place_selection)
          place_final = place_rec.iloc[place_selection[0] - 1,:]
          break
        except:
          print("제대로 된 번호를 입력해주세요! (예시: 3번)\n")

      loc_x = place_final['위도']
      loc_y = place_final['경도']


print("\n마지막으로 아직 아쉽다면 마무리로 술집은 어떠신가요? 가고 싶은 술집에 대해 자유롭게 적어주세요 \n(술집 추천을 원하지 않으시면 X를 입력해주세요)\n")

while(1):
  bar_sentence = input()
  if (bar_sentence == 'X' or bar_sentence == 'x'):
      flagBar = 0
      break
  else:
    try:
      keywords_main, keywords_sub, exclude_keywords, Islike = extract_keywords(bar_sentence)
      bar_rec = recommend_rows_based_on_keywords(df=bar, model=model, keywords_main=keywords_main, keywords_sub=keywords_sub, exclude_keywords=exclude_keywords, Islike=Islike, \
                                                  loc_x=loc_x, loc_y=loc_y)
      break
    except:
      print("죄송합니다. 문장을 이해할 수 없습니다. 다시 입력해주세요.\n")

if flagBar == 1:
    print("\n말씀해주신 결과를 바탕으로 다음과 같은 장소들을 찾아보았습니다! 마음에 드는 번호를 입력해주세요\n")

    place_name =[]
    for i in range(len(bar_rec)):
        place_name = bar_rec.iloc[i, :]['장소명']
        menu_name = bar_rec.iloc[i, :]['대표메뉴'].split(',', 2)[:2]
        score = bar_rec.iloc[i, :]['final_score']
        print("[{}번] 장소명: {}, 대표메뉴: {}".format(i+1, place_name, menu_name))
        i = i + 1
    if len(place_name) == 0 :
      print('죄송합니다ㅠㅠ 조건을 만족하는 추천지가 없습니다!\n')

    else :
      while(1):
        try:
          bar_selection = input()
          bar_selection = extract_integers(bar_selection)
          bar_final = bar_rec.iloc[bar_selection[0] - 1,:]
          break
        except:
          print("제대로 된 번호를 입력해주세요! (예시: 3번)\n")

      loc_x = bar_final['위도']
      loc_y = bar_final['경도']


print("\n감사합니다~ 오늘 일정에 대해 정리해드리겠습니다!\n")
rec_list = []

# 선택한 일정 체크
if food_final is not None and not food_final.empty:
    rec_list.append("식당")

if cafe_final is not None and not cafe_final.empty:
    rec_list.append("카페")

if place_final is not None and not place_final.empty:
    rec_list.append("장소")

if bar_final is not None and not bar_final.empty:
    rec_list.append("술집")


# rec_list에 담긴 각 항목에 대해 처리
for rec in rec_list:
    # rec가 "식당"일 경우
    if rec == "식당" and not food_final.empty:
        row = food_final.copy()  # food_final 데이터프레임의 첫 번째 행
        menus = row['대표메뉴']
        print(f"{rec}: {row['장소명']} / 대표메뉴: {menus} / 주소: {row['주소']}")

    # rec가 "카페"일 경우
    elif rec == "카페" and not cafe_final.empty:
        row = cafe_final.copy()  # cafe_final 데이터프레임의 첫 번째 행
        menus = row['대표메뉴']
        print(f"{rec}: {row['장소명']} / 대표메뉴: {menus} / 주소: {row['주소']}")

    # rec가 "장소"일 경우
    elif rec == "장소" and not place_final.empty:
        row = place_final.copy()  # place_final 데이터프레임의 첫 번째 행
        print(f"{rec}: {row['장소명']} / 주소: {row['주소']}")

    # rec가 "술집"일 경우
    elif rec == "술집" and not bar_final.empty:
        row = bar_final.copy()  # bar_final 데이터프레임의 첫 번째 행
        menus = row['대표메뉴']
        print(f"{rec}: {row['장소명']} / 대표메뉴: {menus} / 주소: {row['주소']}")

안녕하세요! 멋진 하루를 보낼 수 있도록 도와드리겠습니다!

오늘 어디를 방문하실 계획인가요? 방문 계획이 있다면 인근 지하철 역을 말씀해주시겠어요? 
(지하철 역 입력을 원하지 않으시면 X를 입력해주세요)

잠실역

감사합니다! 그럼 이제 일정을 세워보겠습니다!
금강산도 식후경! 우선 밥을 먹는 것은 어떨까요? 드시고 싶은 음식의 느낌을 자유롭게 적어주세요. 
(식당 추천을 원하지 않으시면 X를 입력해주세요)

따뜻하고 든든한 음식 없을까? 양식이나 한식같은거

말씀해주신 결과를 바탕으로 다음과 같은 식당들을 찾아보았습니다! 마음에 드는 번호를 입력해주세요

[1번] 장소명: greasy house, 대표메뉴: ['갈릭 햄버그 스테이크', ' 슬로피조 햄버그 스테이크']
[2번] 장소명: 블랭크, 대표메뉴: ['블랭크 크로플', ' 라비나치즈케이크']
[3번] 장소명: 고고생고기, 대표메뉴: ['고고모둠', ' 하이포크 생삼겹 ']
[4번] 장소명: 다샤브 송파점, 대표메뉴: ['샤브샤브 인세트', ' 샤브샤브 얼큰커플세트']
[5번] 장소명: 경리단길 미트파이(잠실점), 대표메뉴: ['비프치즈파이 ', ' 치킨치즈파이 ']
[6번] 장소명: 오브제, 대표메뉴: ['딸기듬뿍 생딸기케이크', ' 로투스치즈케이크']
[7번] 장소명: 맘스터치 송파마천점, 대표메뉴: ['탄두리싸이버거', ' 마라싸이버거']
[8번] 장소명: 맘스터치, 대표메뉴: ['탄두리싸이버거', ' 마라싸이버거']
[9번] 장소명: 터치, 대표메뉴: ['탄두리싸이버거', ' 마라싸이버거']
[10번] 장소명: 맘스터치 위례중앙점, 대표메뉴: ['탄두리싸이버거', ' 마라싸이버거']
1번

느낌있는 카페는 어떠신가요? 카페에 대해 자유롭게 적어주세요 
(카페 추천을 원하지 않으시면 X를 입력해주세요)

조용하고 디저트가 맛있는 카페

말씀해주신 결과를 바탕으로 다음과 같은 카페들을 찾아보았습니다! 마음에 드는 번호를 입력해주세요

[1번] 장소명: 엘스카페, 대표메뉴: ['얼그레이 쉬폰케이크 

**※ 지원하는 지역 리스트 : [송파구, 중구, 종로구, 용산구, 강남구, 서초구, 서대문구, 동대문구, 마포구, 영등포구, 성동구, 용산구, 광진구]**

**+ 문장에 포함된 (잠재 + 메인) 키워드의 수가 늘어날수록 탐색 시간도 길어집니다!**

**테스트에 참고해주세요!**